In [2]:
import os

In [4]:
%pwd

'c:\\Users\\Victus\\Documents\\PROJECTS\\DL Project'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: Path
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_training_config(self)-> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size = params.IMAGE_SIZE
        )
        return training_config


In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from keras.optimizers import Adam

In [18]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

        self.optimizer = Adam()  # Create an optimizer instance
        self.model.compile(optimizer=self.optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])  # Compile the model
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
    


In [19]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-10-07 22:48:41,304: INFO: common yaml file: config\config.yaml loaded successfully]
[2024-10-07 22:48:41,319: INFO: common yaml file: params.yaml loaded successfully]
[2024-10-07 22:48:41,323: INFO: common created directory at: artifacts]
[2024-10-07 22:48:41,325: INFO: common created directory at: artifacts\training]
[2024-10-07 22:48:41,805: WARNING: saving_utils Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.7241 - loss: 0.6645 - val_accuracy: 0.8500 - val_loss: 0.2159
Epoch 2/200
 1/23 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.8125 - loss: 0.3962

C:\Users\Victus\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.8125 - loss: 0.3962 - val_accuracy: 1.0000 - val_loss: 0.1486
Epoch 3/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9249 - loss: 0.1912 - val_accuracy: 1.0000 - val_loss: 0.0355
Epoch 4/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - accuracy: 0.9375 - loss: 0.1211 - val_accuracy: 1.0000 - val_loss: 0.0296
Epoch 5/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.9537 - loss: 0.1083 - val_accuracy: 1.0000 - val_loss: 0.0924
Epoch 6/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 1.0000 - loss: 0.0374 - val_accuracy: 1.0000 - val_loss: 0.0378
Epoch 7/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.9783 - loss: 0.0574 - val_accuracy: 1.0000 - val_loss: 0.0175
Epoch 8/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 1.0000 - loss: 0.0151 - val_accuracy: 1.0000 - val_loss: 0.0110
Epoch 9/200
23/23 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.9874 - loss: 0.0460 - val_accuracy: 1.0000 - val_loss: